In [1]:
import os
import glob
import json
import re
from PIL import Image

import pandas as pd
import numpy as np
import matplotlib as plt

In [2]:
def show_files(directory):
    arr = os.listdir(directory)
    arr = [item for item in arr if '.' not in item[0]] # hide hidden directories
    print(directory, "contains:", arr)
    return arr

In [3]:
# location of USB drive on Mac
usb_dir = "/Volumes/DATASETS" 

show_files(usb_dir)
print('\n')

show_files(usb_dir + '/COCO-text')
print('\n')

arr = show_files(usb_dir + '/textOCR')
for item in arr:
    show_files(usb_dir + '/textOCR/' + item) 

/Volumes/DATASETS contains: ['textOCR', 'COCO-text']


/Volumes/DATASETS/COCO-text contains: ['cocotext.v2.json', 'train2014']


/Volumes/DATASETS/textOCR contains: ['test', 'train + validation']
/Volumes/DATASETS/textOCR/test contains: ['test_images']
/Volumes/DATASETS/textOCR/train + validation contains: ['TextOCR_0.1_train.json', 'train', 'TextOCR_0.1_val.json']


In [4]:
data = json.load(open(usb_dir + '/COCO-text/cocotext.v2.json')) # Get JSON data
annotations = data['anns'].values() # Get only the annotations as dictionary

coco = pd.DataFrame(annotations) # Convert dictionary to dataframe
coco.head()

,mask,class,bbox,image_id,id,language,area,utf8_string,legibility
0,"[468.9, 286.7, 468.9, 295.2, 493.0, 295.8, 493...",machine printed,"[468.9, 286.7, 24.1, 9.1]",217925,45346,english,206.06,New,legible
1,"[344.5, 261.5, 348.1, 261.5, 348.2, 263.4, 344...",machine printed,"[344.5, 261.5, 3.7, 1.9]",483569,153036,english,6.93,,illegible
2,"[362.4, 280.9, 359.2, 286.3, 367.1, 291.0, 369...",machine printed,"[359.2, 280.9, 10.7, 10.1]",417153,125303,english,57.09,,illegible
3,"[570.9, 9.6, 570.9, 14.3, 557.1, 14.3, 556.8, ...",machine printed,"[556.8, 9.4, 14.1, 4.9]",15451,21639,english,66.94,,illegible
4,"[489.4, 131.8, 502.7, 135.6, 502.7, 156.4, 493...",machine printed,"[489.4, 131.8, 13.3, 24.6]",379024,112792,english,249.08,W,legible


In [5]:
len(coco)

201126

In [6]:
data = json.load(open(usb_dir + '/textOCR/train + validation/TextOCR_0.1_train.json'))
annotations = data['anns'].values()

ocr_train = pd.DataFrame(annotations)
ocr_train.head()

,id,image_id,bbox,utf8_string,points,area
0,a4ea732cd3d5948a_1,a4ea732cd3d5948a,"[525.83, 3.4, 197.64, 33.94]",Performance,"[525.83, 3.4, 723.47, 7.29, 722.76, 36.99, 525...",6707.90
1,a4ea732cd3d5948a_2,a4ea732cd3d5948a,"[534.67, 64.68, 91.22, 38.19]",Sport,"[535.73, 64.68, 623.41, 67.51, 625.89, 102.87,...",3483.69
2,a4ea732cd3d5948a_3,a4ea732cd3d5948a,"[626.95, 63.62, 96.52, 31.82]",Watch,"[626.95, 63.62, 721.7, 63.62, 723.47, 95.44, 6...",3071.27
3,a4ea732cd3d5948a_4,a4ea732cd3d5948a,"[577.4, 141.87, 147.13, 43.1]",...period.,"[580.02, 143.61, 724.53, 141.87, 723.66, 184.9...",6341.30
4,a4ea732cd3d5948a_5,a4ea732cd3d5948a,"[391.03, 163.9, 60.82, 38.65]",.,"[395.2, 163.9, 451.85, 191.94, 445.59, 202.55,...",2350.69


In [7]:
data = json.load(open(usb_dir + '/textOCR/train + validation/TextOCR_0.1_val.json'))
annotations = data['anns'].values()

ocr_val = pd.DataFrame(annotations)
ocr_val.head()

,id,image_id,bbox,utf8_string,points,area
0,a7ad2bcb93d48576_1,a7ad2bcb93d48576,"[76.73, 63.84, 141.41, 30.66]",RICHARD,"[77.3, 63.84, 217.0, 64.4, 218.14, 94.5, 76.73...",4335.63
1,a7ad2bcb93d48576_2,a7ad2bcb93d48576,"[41.53, 93.93, 206.14, 60.2]",moRGAn,"[42.09, 93.93, 245.39, 93.93, 247.67, 152.99, ...",12409.63
2,a7ad2bcb93d48576_3,a7ad2bcb93d48576,"[39.82, 183.09, 207.28, 75.53]",ALTERED,"[42.66, 186.5, 241.99, 183.09, 247.1, 258.62, ...",15655.86
3,a7ad2bcb93d48576_4,a7ad2bcb93d48576,"[41.53, 251.24, 208.98, 88.59]",CARBOn,"[42.09, 251.24, 245.96, 251.24, 250.51, 335.28...",18513.54
4,a7ad2bcb93d48576_5,a7ad2bcb93d48576,"[46.46, 158.99, 16.73, 9.74]",'An,"[46.46, 158.99, 61.98, 159.3, 63.19, 168.73, 4...",162.95


In [8]:
# Merge OCR dataframes
ocr = pd.concat([ocr_val, ocr_train])
len(ocr)

1202339

In [9]:
# For merging dataframes

# Source identifier for easier image location (can be dropped if unnecessary)
# coco['src'] = 'COCO-Text'
# ocr['src'] = 'TextOCR'

# Give TextOCR language tag (assuming all data is English)
ocr['language'] = 'english'

# Drop all rows where the text is unidentifiable (also makes legibility redundant)
coco = coco[coco['legibility'] == 'legible']
coco.drop(columns=['legibility'], inplace=True)

coco = coco[coco['utf8_string'] != '']
ocr = ocr[ocr['utf8_string'] != '.']

# Drop class; being machine printed or not is irrelevant
coco.drop(columns=['class'], inplace=True)

# Rename columns to match
ocr.rename(columns = {'points':'mask'}, inplace=True)

# Reorder columns
cols = ['id', 'image_id', 'mask', 'bbox', 'utf8_string', 'language', 'area']
coco = coco[cols]
ocr = ocr[cols]

In [10]:
coco.head()

,id,image_id,mask,bbox,utf8_string,language,area
0,45346,217925,"[468.9, 286.7, 468.9, 295.2, 493.0, 295.8, 493...","[468.9, 286.7, 24.1, 9.1]",New,english,206.06
4,112792,379024,"[489.4, 131.8, 502.7, 135.6, 502.7, 156.4, 493...","[489.4, 131.8, 13.3, 24.6]",W,english,249.08
6,8231,122908,"[577.8, 148.3, 601.7, 149.1, 611.0, 196.2, 575...","[575.7, 148.3, 35.3, 47.9]",NAVY,english,1366.38
13,175298,544815,"[398.7, 16.5, 385.0, 18.2, 387.1, 48.7, 401.7,...","[385.0, 16.5, 16.7, 32.2]",6,english,426.05
15,125471,417556,"[385.2, 53.8, 383.9, 66.2, 393.8, 67.1, 393.8,...","[383.9, 53.8, 17.9, 13.3]",$189,english,162.18


In [11]:
ocr.head()

,id,image_id,mask,bbox,utf8_string,language,area
0,a7ad2bcb93d48576_1,a7ad2bcb93d48576,"[77.3, 63.84, 217.0, 64.4, 218.14, 94.5, 76.73...","[76.73, 63.84, 141.41, 30.66]",RICHARD,english,4335.63
1,a7ad2bcb93d48576_2,a7ad2bcb93d48576,"[42.09, 93.93, 245.39, 93.93, 247.67, 152.99, ...","[41.53, 93.93, 206.14, 60.2]",moRGAn,english,12409.63
2,a7ad2bcb93d48576_3,a7ad2bcb93d48576,"[42.66, 186.5, 241.99, 183.09, 247.1, 258.62, ...","[39.82, 183.09, 207.28, 75.53]",ALTERED,english,15655.86
3,a7ad2bcb93d48576_4,a7ad2bcb93d48576,"[42.09, 251.24, 245.96, 251.24, 250.51, 335.28...","[41.53, 251.24, 208.98, 88.59]",CARBOn,english,18513.54
4,a7ad2bcb93d48576_5,a7ad2bcb93d48576,"[46.46, 158.99, 61.98, 159.3, 63.19, 168.73, 4...","[46.46, 158.99, 16.73, 9.74]",'An,english,162.95


In [12]:
# Finally merge the two dataframes
df = pd.concat([coco, ocr])

# Shuffle the rows
df = df.sample(frac = 1)
df.head(20)

,id,image_id,mask,bbox,utf8_string,language,area
187669,011c88f89ef53e23_6,011c88f89ef53e23,"[542.72, 567.08, 549.92, 567.08, 550.12, 573.1...","[542.52, 567.08, 7.6, 6.7]",FC,english,50.92
597775,d21b4ee795fda2c4_66,d21b4ee795fda2c4,"[711.38, 497.56, 710.92, 509.7, 699.01, 505.58...","[699.01, 493.89, 12.37, 15.81]",N,english,195.57
684849,261fe090b52b7034_1,261fe090b52b7034,"[198.83, 159.62, 463.95, 144.15, 462.66, 191.4...","[197.11, 144.15, 266.84, 58.44]",SNOWboarding,english,15594.13
822824,4742281b5d90fae7_24,4742281b5d90fae7,"[312.55, 215.38, 359.45, 214.48, 359.25, 222.6...","[312.55, 214.48, 46.9, 8.6]",CHAIGNRAU,english,403.34
750305,2d25ba63d73ed807_22,2d25ba63d73ed807,"[561.76, 362.43, 604.48, 360.22, 605.21, 398.1...","[561.76, 360.22, 43.45, 38.67]",14g,english,1680.21
200004,018c95ebb9fece08_22,018c95ebb9fece08,"[596.88, 759.81, 654.32, 768.17, 653.68, 780.3...","[596.45, 759.81, 57.87, 20.58]",medicamento,english,1190.96
1696,58f38a13d56c2f31_13,58f38a13d56c2f31,"[250.96, 304.2, 286.81, 305.29, 285.73, 337.88...","[249.88, 304.2, 36.93, 33.68]",H,english,1243.80
507246,170ecbaec3a711c1_131,170ecbaec3a711c1,"[526.13, 191.24, 550.94, 205.26, 550.94, 214.6...","[526.13, 191.24, 24.81, 23.44]",PRACTICE,english,581.55
122527,5886a29dbfbc84d8_11,5886a29dbfbc84d8,"[286.04, 394.27, 413.32, 379.21, 417.77, 416.5...","[286.04, 379.21, 131.73, 56.12]",OYSTER,english,7392.69
168388,01f16152d32ca714_101,01f16152d32ca714,"[785.17, 215.2, 860.55, 214.75, 861.23, 248.35...","[785.17, 214.75, 76.06, 33.83]",?△,english,2573.11


In [13]:
def show_image (image_id):

    coco_dir = '/COCO-text/train2014/COCO_train2014_'
    ocr_dir = '/textOCR/train + validation/train/'
    
    if re.search('[a-zA-Z]', str(image_id)) == None: # COCO-text image ID is an integer!
        image = Image.open(usb_dir + coco_dir + '{}'.format(image_id).zfill(12) + '.jpg')
    else: # TextOCR image ID is a string!
        image = Image.open(usb_dir + ocr_dir + '{}.jpg'.format(image_id))
        
    image.show()

In [14]:
show_image (217925)

In [15]:
show_image ('a7ad2bcb93d48576')